In [29]:
import geopandas as gpd
from datetime import datetime
from libcomcat.search import search
import pandas as pd
import pprint
import os

# Output dir

In [41]:
# Crée le dossier principal s'il n'existe pas
os.makedirs("data_input/USGS", exist_ok=True)

# Import

In [16]:

# =============================================================================
# =============================================================================

# --- 1. Paramètres utilisateur ---
geojson_path = "myanmar.json"    # <- à adapter
date_start = "2025-03-22"   # au format YYYY-MM-DD
date_end   = "2025-04-30"
# =============================================================================
# =============================================================================



# --- 2. Charger le GeoJSON et s'assurer du bon CRS ---
gdf = gpd.read_file(geojson_path)
if gdf.crs is not None and gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(4326)
minx, miny, maxx, maxy = gdf.total_bounds

# --- 3. Conversion des dates ---
starttime = datetime.fromisoformat(date_start)
endtime = datetime.fromisoformat(date_end)

# --- 4. Recherche des événements ---
events = search(
    minlongitude=minx,
    minlatitude=miny,
    maxlongitude=maxx,
    maxlatitude=maxy,
    starttime=starttime,
    endtime=endtime,
)

# --- 5. Afficher les attributs du premier événement pour inspecter ---
if events:
    first = events[0]
    print(first)
    print("----------- Attributs disponibles ----------")
    print(dir(first))
else:
    print("Aucun événement trouvé.")

# --- 6. Construire le DataFrame selon les attributs disponibles ---
# Liste possible des attributs à récupérer
cols = ["id", "time", "magnitude", "depth", "latitude", "longitude", "magtype", "gap", "rms", "horizontal_error", "vertical_error"]

data = []
for e in events:
    d = {}
    for c in cols:
        d[c] = getattr(e, c, None)
    data.append(d)
df = pd.DataFrame(data)
print(df.head())

# Tu pourras adapter la liste 'cols' en fonction de ce que tu vois dans le dir(first)


us6000q3p6 2025-03-24 04:38:32.700000 (29.189,93.747) 10.0 km M4.2
----------- Attributs disponibles ----------
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_jdict', 'alert', 'depth', 'getDetailEvent', 'getDetailURL', 'hasProduct', 'hasProperty', 'id', 'latitude', 'location', 'longitude', 'magnitude', 'properties', 'time', 'toDict', 'url']
           id                             time  magnitude  depth  latitude  \
0  us6000q3p6 2025-03-24 04:38:32.700000+00:00        4.2   10.0   29.1888   
1  us6000q3ny 2025-03-24 04:43:00.743000+00:00        4.1   10.0   29.2119   
2  us7000pm7w 2025-03-24 11:21:38.311000+00:00        5.0   10.0   28.1045   
3  us7000pmg7 

In [17]:
df = df.sort_values("magnitude", ascending=False)
df

,id,time,magnitude,depth,latitude,longitude,magtype,gap,rms,horizontal_error,vertical_error
5,us7000pn9s,2025-03-28 06:20:52.715000+00:00,7.7,10.0,22.0110,95.9363,None,None,None,None,None
6,us7000pn9z,2025-03-28 06:32:04.777000+00:00,6.7,10.0,21.6975,95.9690,None,None,None,None,None
66,us6000q5ps,2025-04-13 02:24:57.717000+00:00,5.3,10.0,21.2292,96.0897,None,None,None,None,None
78,us7000pv6f,2025-04-26 04:21:49.088000+00:00,5.1,10.0,31.0900,98.9820,None,None,None,None,None
30,us7000pnkj,2025-03-29 09:20:48.229000+00:00,5.1,10.0,19.6858,96.0761,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
64,us6000q5cs,2025-04-11 10:52:56.066000+00:00,4.1,10.0,24.0430,91.3615,None,None,None,None,None
59,us6000q575,2025-04-10 16:17:58.886000+00:00,3.9,10.0,21.7193,95.9475,None,None,None,None,None
56,us6000q4x4,2025-04-09 14:40:17.442000+00:00,3.6,10.0,21.4447,95.9352,None,None,None,None,None
57,us6000q4xc,2025-04-09 15:59:18.014000+00:00,3.6,10.0,21.9457,96.0058,None,None,None,None,None


## Création d'un geo df

In [18]:

import geopandas as gpd
from shapely.geometry import Point

gdf_map = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)],
    crs="EPSG:4326"
)


In [20]:
# Export du GDF

gdf_map.to_file("data_input/USGS/seismes_request.gpkg", layer="seismes", driver="GPKG")


### Earthquakes initial querie - Show on map 

In [21]:
import folium
import numpy as np
from branca.colormap import linear

# Fonction pour rayon basé sur la magnitude
def mag_to_radius(mag):
    return 2 * np.exp(mag - 4)

# Définir l'échelle des magnitudes
min_mag = df["magnitude"].min()
max_mag = df["magnitude"].max()

# Création de la carte
m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=6, tiles="cartodbpositron")

# Colormap jaune -> rouge
colormap = linear.YlOrRd_09.scale(min_mag, max_mag)
colormap.caption = "Magnitude"
colormap.add_to(m)

# Boucle sur les événements
for _, row in df.iterrows():
    mag = row["magnitude"]
    radius = mag_to_radius(mag) if mag else 2
    fill_color = colormap(mag) if mag else "#cccccc"

    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=radius,
        popup=f"Mag {mag}<br>ID: {row['id']}",
        color="black",              # Bordure noire
        weight=1,                   # Épaisseur de la bordure
        fill=True,
        fill_color=fill_color,
        fill_opacity=0.6
    ).add_to(m)

m  # Affiche la carte dans le notebook


# Earthquake datapoint selection with ID

In [22]:
event_id = df.iloc[0]["id"]
print(event_id)  # Ex: 'us6000kikc'


us7000pn9s


### Accès à toutes les données de Shakemap

In [23]:
import requests
import json

event_id = "us7000pn9s"
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid={event_id}&format=geojson"
r = requests.get(url)
data = r.json()
data

{'type': 'Feature',
 'properties': {'mag': 7.7,
  'place': '2025 Mandalay, Burma (Myanmar) Earthquake',
  'time': 1743142852715,
  'updated': 1751393086583,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000pn9s',
  'felt': 2531,
  'cdi': 9.1,
  'mmi': 9.953,
  'alert': 'red',
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 2910,
  'net': 'us',
  'code': '7000pn9s',
  'ids': ',us7000pn9s,usauto7000pn9s,pt25087002,at00sttln4,',
  'sources': ',us,usauto,pt,at,',
  'types': ',dyfi,earthquake-name,finite-fault,general-text,ground-failure,impact-text,internal-moment-tensor,internal-origin,losspager,moment-tensor,oaf,origin,phase-data,shakemap,',
  'nst': 246,
  'dmin': 3.474,
  'rms': 0.89,
  'gap': 24,
  'magType': 'mww',
  'type': 'earthquake',
  'title': 'M 7.7 - 2025 Mandalay, Burma (Myanmar) Earthquake',
  'products': {'dyfi': [{'indexid': '26646279',
     'indexTime': 1751393091187,
     'id': 'urn:usgs-product:us:dyfi:us7000pn9s:1751393086583',
     'ty

In [24]:
from datetime import datetime
properties = data.get("properties", {})
products = properties.get("products", {})

files = []

for product_type, product_list in products.items():
    for product in product_list:
        contents = product.get('contents', {})
        if isinstance(contents, dict):
            for file_name, file_info in contents.items():
                # Récupérer le timestamp s'il existe
                lastmod = file_info.get("lastModified")
                # Conversion si présent
                if lastmod is not None:
                    # lastmod est en millisecondes
                    lastmod_str = datetime.utcfromtimestamp(int(lastmod) / 1000).strftime('%Y-%m-%d %H:%M:%S')
                else:
                    lastmod_str = "unknown"
                files.append({
                    "product_type": product_type,
                    "file_name": file_name,
                    "url": file_info.get("url"),
                    "size": file_info.get("length", "unknown"),
                    "content_type": file_info.get("contentType", "unknown"),
                    "last_modified": lastmod_str
                })

files = sorted(files, key=lambda x: (x['product_type'], x['file_name']))

# Printing products

for f in files:
    print(
        f"{f['product_type']:16} {f['file_name']:40} "
        f"{f['size']:>10} bytes  {f['last_modified']:19}  {f['url']}"
    )

products_listing = []

for f in files:
    products_listing.append({
        "product_type": f["product_type"],
        "file_name": f["file_name"],
        "size": f["size"],
        "last_modified": f["last_modified"],
        "url": f["url"]
    })

df_product = pd.DataFrame(products_listing)

# products_listing

dyfi             cdi_geo.txt                                   33462 bytes  2025-07-01 18:04:46  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751393086583/cdi_geo.txt
dyfi             cdi_geo.xml                                  110595 bytes  2025-07-01 18:04:46  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751393086583/cdi_geo.xml
dyfi             cdi_geo_1km.txt                              115242 bytes  2025-07-01 18:04:46  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751393086583/cdi_geo_1km.txt
dyfi             cdi_zip.txt                                   23086 bytes  2025-07-01 18:04:46  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751393086583/cdi_zip.txt
dyfi             cdi_zip.xml                                   48599 bytes  2025-07-01 18:04:46  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751393086583/cdi_zip.xml
dyfi             contents.xml                                   4224 bytes  2025-07-01 18:04:46  https://e

/var/folders/8b/jbf_9gw94xs45f1kt0j8k5880000gq/T/ipykernel_80699/4120806721.py:17: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  lastmod_str = datetime.utcfromtimestamp(int(lastmod) / 1000).strftime('%Y-%m-%d %H:%M:%S')


In [25]:
df_product.to_csv(f"data_input/USGS/products_id_{event_id}.csv")

In [26]:
df_product

,product_type,file_name,size,last_modified,url
0,dyfi,cdi_geo.txt,33462,2025-07-01 18:04:46,https://earthquake.usgs.gov/product/dyfi/us700...
1,dyfi,cdi_geo.xml,110595,2025-07-01 18:04:46,https://earthquake.usgs.gov/product/dyfi/us700...
2,dyfi,cdi_geo_1km.txt,115242,2025-07-01 18:04:46,https://earthquake.usgs.gov/product/dyfi/us700...
3,dyfi,cdi_zip.txt,23086,2025-07-01 18:04:46,https://earthquake.usgs.gov/product/dyfi/us700...
4,dyfi,cdi_zip.xml,48599,2025-07-01 18:04:46,https://earthquake.usgs.gov/product/dyfi/us700...
...,...,...,...,...,...
176,shakemap,download/shake_result.hdf,31963388,2025-06-06 13:36:22,https://earthquake.usgs.gov/product/shakemap/u...
177,shakemap,download/shakemap.kmz,831540,2025-06-06 13:36:22,https://earthquake.usgs.gov/product/shakemap/u...
178,shakemap,download/shape.zip,26264669,2025-06-06 13:36:22,https://earthquake.usgs.gov/product/shakemap/u...
179,shakemap,download/stationlist.json,3499574,2025-06-06 13:36:22,https://earthquake.usgs.gov/product/shakemap/u...


## Access to product - function

In [27]:
import requests

def safe_get_json(df, pattern, strict=True, verbose=True):
    """
    Recherche un fichier JSON contenant `pattern` dans son nom.
    Retourne le contenu JSON ou None si non disponible/erreur.
    - strict=True : lève une erreur si plusieurs fichiers, sinon prend le premier.
    - verbose=True : affiche les warnings.
    """
    mask = df['file_name'].str.contains(pattern, case=False, na=False) & df['file_name'].str.endswith('.json')
    matches = df[mask]
    if len(matches) == 0:
        if verbose:
            print(f"[SAFE ACCESSOR] Aucun fichier JSON trouvé pour pattern '{pattern}'.")
        return None
    if len(matches) > 1:
        if strict:
            if verbose:
                print(f"[SAFE ACCESSOR] Plusieurs fichiers trouvés pour pattern '{pattern}':")
                print("\n".join(matches['file_name']))
                print("=> Precise le pattern ou utilise strict=False.")
            return None
        else:
            if verbose:
                print(f"[SAFE ACCESSOR] Plusieurs fichiers trouvés, on prend le premier : {matches.iloc[0]['file_name']}")
    file_name = matches.iloc[0]['file_name']
    url = matches.iloc[0]['url']
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        if verbose:
            print(f"[SAFE ACCESSOR] Erreur lors du téléchargement de {file_name}: {e}")
        return None


## Get propreties

Detailed information on the earthquake

In [30]:
json_properties = safe_get_json(df_product, "properties")
if json_properties is not None:
    df_properties = pd.DataFrame(list(json_properties.items()), columns=["key", "value"])
    display(df_properties)
    # Export CSV sans index
    os.makedirs("data_input/USGS", exist_ok=True)
    df_properties.to_csv("data_input/USGS/properties.csv", index=False)
    print("Exported: data_input/USGS/properties.csv")
else:
    print("No properties data.")


,key,value
0,average-rise-time,9.53
1,average-rupture-velocity,3.88
2,crustal-model,1D crustal model interpolated from CRUST2.0 (B...
3,depth,10.0
4,derived-magnitude,7.695717
5,derived-magnitude-type,Mw
6,eventsource,us
7,eventsourcecode,7000pn9s
8,eventtime,2025-03-28T00:00:00.000000Z
9,hypocenter-x,115.0


Exported: data_input/USGS/properties.csv


## Get comments

In [31]:
json_comments = safe_get_json(df_product, "comments")
if json_comments is not None:
    os.makedirs("data_input/USGS", exist_ok=True)
    md_path = "data_input/USGS/comments.md"
    with open(md_path, "w", encoding="utf-8") as f:
        for key, value in json_comments.items():
            title = key.replace('_', ' ').capitalize()
            f.write(f"### {title}\n\n{value}\n\n")
    print(f"Exported: {md_path}")
else:
    print("No comments data.")

Exported: data_input/USGS/comments.md


### Shake intensity category - df for category and german and french translations

In [32]:
import os
import pandas as pd

data_intensity_category = [
    [1,   "Not felt",    "None",           "Nicht fühlbar",   "Keine",         "Non ressenti",     "Aucun"],
    [2,   "Weak",        "None",           "Schwach",         "Keine",         "Faible",           "Aucun"],
    [3,   "Weak",        "None",           "Schwach",         "Keine",         "Faible",           "Aucun"],
    [4,   "Light",       "None",           "Leicht",          "Keine",         "Léger",            "Aucun"],
    [5,   "Moderate",    "Very light",     "Mässig",          "Sehr gering",   "Modéré",           "Très léger"],
    [6,   "Strong",      "Light",          "Stark",           "Gering",        "Fort",             "Léger"],
    [7,   "Very strong", "Moderate",       "Sehr stark",      "Mittel",        "Très fort",        "Modéré"],
    [8,   "Severe",      "Moderate/heavy", "Heftig",          "Mittel/hoch",   "Sévère",           "Modéré/important"],
    [9,   "Violent",     "Heavy",          "Heftig",          "Hoch",          "Violent",          "Important"],
    [10,  "Extreme",     "Very heavy",     "Extrem",          "Sehr hoch",     "Extrême",          "Très important"]
]

df_intensity_cat = pd.DataFrame(
    data_intensity_category,
    columns=["Intensity", "Shaking_EN", "Damage_EN", "Shaking_DE", "Damage_DE", "Shaking_FR", "Damage_FR"]
)

output_dir = "data_input/USGS"
os.makedirs(output_dir, exist_ok=True)
outpath = f"{output_dir}/mmi_intensity_category.csv"

df_intensity_cat.to_csv(outpath, index=False)
print(f"✅ Exported: {outpath}")

df_intensity_cat


✅ Exported: data_input/USGS/mmi_intensity_category.csv


,Intensity,Shaking_EN,Damage_EN,Shaking_DE,Damage_DE,Shaking_FR,Damage_FR
0,1,Not felt,None,Nicht fühlbar,Keine,Non ressenti,Aucun
1,2,Weak,None,Schwach,Keine,Faible,Aucun
2,3,Weak,None,Schwach,Keine,Faible,Aucun
3,4,Light,None,Leicht,Keine,Léger,Aucun
4,5,Moderate,Very light,Mässig,Sehr gering,Modéré,Très léger
5,6,Strong,Light,Stark,Gering,Fort,Léger
6,7,Very strong,Moderate,Sehr stark,Mittel,Très fort,Modéré
7,8,Severe,Moderate/heavy,Heftig,Mittel/hoch,Sévère,Modéré/important
8,9,Violent,Heavy,Heftig,Hoch,Violent,Important
9,10,Extreme,Very heavy,Extrem,Sehr hoch,Extrême,Très important


## Get shapefiles

In [33]:
import os
import requests
import zipfile
import geopandas as gpd
from shapely.geometry import box
import cartopy.io.shapereader as shpreader

def try_download_and_extract_shp(df_product, pattern="download/shape.zip", base_dir="data_output/shape_mmi", shapefile_name="mi.shp", verbose=True):
    """Télécharge et extrait le .shp si dispo. Retourne un GeoDataFrame ou None."""
    # Vérifie si la ressource existe dans la liste de fichiers
    mask = df_product["file_name"] == pattern
    if not mask.any():
        if verbose:
            print(f"[INFO] Pas de fichier '{pattern}' trouvé dans df_product.")
        return None
    mmi_url = df_product.loc[mask, "url"].iloc[0]
    os.makedirs(base_dir, exist_ok=True)
    zip_path = os.path.join(base_dir, "mmi.zip")
    try:
        # 1. Télécharger le ZIP
        with requests.get(mmi_url, stream=True) as r:
            r.raise_for_status()
            with open(zip_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        # 2. Extraire .shp (+ nécessaires)
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            members = [n for n in zip_ref.namelist() if n.endswith(('.shp','.shx','.dbf','.prj'))]
            zip_ref.extractall(base_dir, members=members)
        os.remove(zip_path)
        # 3. Lister les .shp extraits (facultatif)
        shp_files = []
        for root, dirs, files in os.walk(base_dir):
            for file in files:
                if file.endswith(".shp"):
                    rel_path = os.path.relpath(os.path.join(root, file), base_dir)
                    shp_files.append(rel_path)
        if verbose:
            print("Shapefiles extraits :")
            for path in shp_files:
                print("-", path)
        # 4. Charger le .shp demandé
        shp_path = os.path.join(base_dir, shapefile_name)
        if not os.path.exists(shp_path):
            print(f"[WARNING] Shapefile '{shapefile_name}' non trouvé.")
            return None
        gdf = gpd.read_file(shp_path)
        return gdf
    except Exception as e:
        print(f"[ERROR] Extraction échouée : {e}")
        return None

def get_natural_earth(category, name, resolution="10m"):
    """Retourne un GeoDataFrame d'une couche Natural Earth."""
    shp_path = shpreader.natural_earth(resolution=resolution, category=category, name=name)
    gdf = gpd.read_file(shp_path)
    return gdf

# === Main workflow (safe) ===

gdf = try_download_and_extract_shp(df_product, pattern="download/shape.zip", base_dir="data_output/shape_mmi", shapefile_name="mi.shp")

if gdf is not None:
    print("Aperçu du MMI shapefile :", gdf.head())
    # (1) Merge toutes les géométries
    merged = gdf.geometry.union_all()
    mmi_true_extent = merged.envelope
    mmi_gdf_extent = gpd.GeoDataFrame({"name": ["MMI_true_extent"]}, geometry=[mmi_true_extent], crs=gdf.crs)
    # (2) Charger Natural Earth oceans/countries (jamais d’erreur, c’est local)
    gdf_ocean = get_natural_earth("physical", "ocean", "10m")
    gdf_countries = get_natural_earth("cultural", "admin_0_countries", "10m")
    # (3) Clipper : enlever l’océan du MMI
    ocean_union = gdf_ocean.unary_union
    gdf_mmi_land = gdf.copy()
    gdf_mmi_land["geometry"] = gdf_mmi_land.geometry.apply(lambda geom: geom.difference(ocean_union))
    gdf_mmi_land = gdf_mmi_land[~gdf_mmi_land.is_empty & gdf_mmi_land.geometry.notnull()]
    # (4) Optionnel : aperçu
    # gdf_mmi_land.explore(column="PARAMVALUE", cmap="OrRd", tiles="CartoDB positron")
    print("MMI clipped to land only. Ready for mapping/analysis.")
else:
    print("Pas de couche MMI disponible.")

import os

def safe_export_gdf(gdf, filename, layer):
    export_dir = "data_input/USGS"
    os.makedirs(export_dir, exist_ok=True)
    out_path = os.path.join(export_dir, filename)
    if gdf is not None and not gdf.empty:
        try:
            gdf.to_file(out_path, layer=layer, driver="GPKG")
            print(f"Exported: {out_path} (layer: {layer})")
        except Exception as e:
            print(f"[Export ERROR] {out_path}: {e}")

# --- Exportation explicite ---

safe_export_gdf(gdf,              "mmi_raw.gpkg",           "mmi_raw")
safe_export_gdf(gdf_mmi_land,     "mmi_land_only.gpkg",     "mmi_land_only")
safe_export_gdf(mmi_gdf_extent,   "mmi_extent.gpkg",        "mmi_extent")
safe_export_gdf(gdf_ocean,        "naturalearth_ocean.gpkg","ocean")
safe_export_gdf(gdf_countries,    "naturalearth_countries.gpkg","countries")


Shapefiles extraits :
- psa3p0.shp
- psa1p0.shp
- psa0p3.shp
- pgv.shp
- pga.shp
- mi.shp
Aperçu du MMI shapefile :    AREA  PERIMETER  PGAPOL_  PGAPOL_ID  GRID_CODE  PARAMVALUE  \
0   NaN        NaN       14         14          0         2.8   
1   NaN        NaN       15         15          0         3.0   
2   NaN        NaN       16         16          0         3.2   
3   NaN        NaN       17         17          0         3.4   
4   NaN        NaN       18         18          0         3.6   

                                            geometry  
0  MULTIPOLYGON (((102.15192 26.75, 102.15032 26....  
1  MULTIPOLYGON (((101.22624 26.775, 101.22608 26...  
2  MULTIPOLYGON (((100.12571 26.775, 100.12539 26...  
3  MULTIPOLYGON (((97.07814 26.725, 97.07574 26.7...  
4  MULTIPOLYGON (((97.60229 26.725, 97.60131 26.7...  


/var/folders/8b/jbf_9gw94xs45f1kt0j8k5880000gq/T/ipykernel_80699/3846911956.py:73: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  ocean_union = gdf_ocean.unary_union


MMI clipped to land only. Ready for mapping/analysis.
Exported: data_input/USGS/mmi_raw.gpkg (layer: mmi_raw)
Exported: data_input/USGS/mmi_land_only.gpkg (layer: mmi_land_only)
Exported: data_input/USGS/mmi_extent.gpkg (layer: mmi_extent)
Exported: data_input/USGS/naturalearth_ocean.gpkg (layer: ocean)
Exported: data_input/USGS/naturalearth_countries.gpkg (layer: countries)


### MMI layer - show on map

## Get cities

In [34]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 1. Safe getter
json_cities = safe_get_json(df_product, "cities")

if json_cities is not None and 'all_cities' in json_cities:
    # 2. Création du DataFrame
    df_cities = pd.DataFrame(json_cities['all_cities'])

    # # 3. Export CSV
    # os.makedirs("data_input/USGS", exist_ok=True)
    # csv_path = "data_input/USGS/cities.csv"
    # df_cities.to_csv(csv_path, index=False)
    # print(f"Exported: {csv_path}")

    # 4. Création du GeoDataFrame
    gdf_cities = gpd.GeoDataFrame(
        df_cities,
        geometry=[Point(xy) for xy in zip(df_cities['lon'], df_cities['lat'])],
        crs="EPSG:4326"
    )

    # 5. Export GeoPackage
    gpkg_path = "data_input/USGS/cities.gpkg"
    gdf_cities.to_file(gpkg_path, layer="cities", driver="GPKG")
    print(f"Exported: {gpkg_path}")

    # 6. Affichage rapide (optionnel)
    display(df_cities.head())
else:
    print("No cities data.")


Exported: data_input/USGS/cities.gpkg


,name,ccode,lat,lon,iscap,pop,mmi,on_map
0,Sagaing,MM,21.87870,95.97965,True,78739,9.380555,0
1,Pyu,MM,18.48130,96.43742,False,40386,9.200000,0
2,Yamethin,MM,20.43189,96.13875,False,59867,9.141667,0
3,Pyinmana,MM,19.73810,96.20742,False,97409,9.116667,0
4,Kyaukse,MM,21.60560,96.13508,False,50480,8.900000,0


## Get exposure

Infos to population exposure at different level of MMI, by countries.

In [37]:
import os
import pandas as pd
from babel import Locale

output_dir = "data_input/USGS"
os.makedirs(output_dir, exist_ok=True)

# Recharge la ressource à chaque fois pour être safe !
json_exposure = safe_get_json(df_product, "exposure")
# ou, selon ton code :
# json_exposure = get_json_data(df_product, "exposure")

# Prépare les objets Locale une seule fois
locale_en = Locale('en')
locale_fr = Locale('fr')
locale_de = Locale('de')

def get_country_names(iso2):
    if iso2 == "ALL":
        return {"en": "Total", "fr": "Total", "de": "Total"}
    code = iso2.upper()
    try:
        return {
            "en": locale_en.territories[code],
            "fr": locale_fr.territories[code],
            "de": locale_de.territories[code],
        }
    except KeyError:
        return {"en": code, "fr": code, "de": code}

if json_exposure is not None:
    mmi = json_exposure['population_exposure']['mmi']
    agg = json_exposure['population_exposure']['aggregated_exposure']

    # Total agrégé (par MMI)
    df_pop_agg = pd.DataFrame({
        'country': 'ALL',
        'mmi': mmi,
        'pop_exposure': agg
    })

    # Par pays (long)
    rows = []
    for country in json_exposure['population_exposure']['country_exposures']:
        for m, exp in zip(mmi, country['exposure']):
            rows.append({
                'country': country['country_code'],
                'mmi': m,
                'pop_exposure': exp
            })
    df_pop_countries = pd.DataFrame(rows)

    # Concat total + pays (long format)
    df_exposure_all = pd.concat([df_pop_agg, df_pop_countries], ignore_index=True)

    # Ajoute noms pays dans 3 langues
    for lang in ["en", "fr", "de"]:
        df_exposure_all[f"country_name_{lang}"] = df_exposure_all["country"].apply(lambda x: get_country_names(x)[lang])

    print(df_exposure_all.head(20))

    outpath = f"{output_dir}/exposure_population_by_country_mmi.csv"
    df_exposure_all.to_csv(outpath, index=False)
    print(f"\n✅ Exporté dans: {outpath}")

else:
    print("No exposure data.")


   country  mmi  pop_exposure country_name_en country_name_fr country_name_de
0      ALL    1             0           Total           Total           Total
1      ALL    2             0           Total           Total           Total
2      ALL    3      32989235           Total           Total           Total
3      ALL    4     163708636           Total           Total           Total
4      ALL    5      24656409           Total           Total           Total
5      ALL    6      20277188           Total           Total           Total
6      ALL    7       8787774           Total           Total           Total
7      ALL    8       3636635           Total           Total           Total
8      ALL    9       5800931           Total           Total           Total
9      ALL   10        414753           Total           Total           Total
10      BD    1             0      Bangladesh      Bangladesh     Bangladesch
11      BD    2             0      Bangladesh      Bangladesh   

## Forecast

Prediction of earthquakes of different magnitudes, at timespan of 1 day, 1 week, 1 year

In [38]:
import os
import pandas as pd

output_dir = "data_input/USGS"
os.makedirs(output_dir, exist_ok=True)

# Si la ressource existe…
json_forecast = safe_get_json(df_product, "forecast")
if json_forecast is not None and "forecast" in json_forecast:
    rows = []
    for period in json_forecast["forecast"]:
        for bin_ in period["bins"]:
            row = {
                "timeStart": period["timeStart"],
                "timeEnd": period["timeEnd"],
                "label": period["label"],
                **bin_,  # magnitude, probability, etc.
            }
            rows.append(row)
    df_forecast = pd.DataFrame(rows)
    df_forecast["timeStart_dt"] = pd.to_datetime(df_forecast["timeStart"], unit="ms")
    df_forecast["timeEnd_dt"] = pd.to_datetime(df_forecast["timeEnd"], unit="ms")
    print(df_forecast.head())
    # Export
    outpath = f"{output_dir}/forecast.csv"
    df_forecast.to_csv(outpath, index=False)
    print(f"\n✅ Exported forecast to {outpath}")
else:
    print("No forecast data.")


       timeStart        timeEnd  label  magnitude  p95minimum  p95maximum  \
0  1751310028715  1751396428715  1 Day        3.0           0           5   
1  1751310028715  1751396428715  1 Day        4.0           0           1   
2  1751310028715  1751396428715  1 Day        5.0           0           0   
3  1751310028715  1751396428715  1 Day        6.0           0           0   
4  1751310028715  1751396428715  1 Day        7.0           0           0   

   probability  median                                     fractileValues  \
0       0.7113       1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1       0.1135       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2       0.0106       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3       0.0003       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4       0.0000       0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                      barPercentages            timeStart_dt  \
0  [29, 32

## Get fault

Faults displacement

In [39]:
import os
import geopandas as gpd

output_dir = "data_input/USGS"
os.makedirs(output_dir, exist_ok=True)

json_rupture = safe_get_json(df_product, "rupture")
rupture_gdf = None

if json_rupture is not None and "features" in json_rupture:
    # Convertit en GeoDataFrame
    rupture_gdf = gpd.GeoDataFrame.from_features(json_rupture["features"])
    print(rupture_gdf.head())
    # Exporte en GeoPackage
    rupture_gdf.to_file(f"{output_dir}/rupture_fault.gpkg", layer="rupture", driver="GPKG")
    print(f"\n✅ Exported rupture GeoDataFrame to {output_dir}/rupture_fault.gpkg")
else:
    print("No rupture data.")


                                            geometry    rupture type
0  MULTIPOLYGON Z (((95.9768 22.4723 0, 95.9751 2...  rupture extent

✅ Exported rupture GeoDataFrame to data_input/USGS/rupture_fault.gpkg


/Users/bajj@mediait.ch/.pyenv/versions/3.12.2/lib/python3.12/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


## Get aftershocks

Per default 30 days

In [40]:
from datetime import datetime, timedelta, timezone
import geopandas as gpd
from shapely.geometry import Point

# =============================================================================
# =============================================================================
# --- 1. Paramètres utilisateur ---
days_after = 30
# =============================================================================
# =============================================================================


# --- 2. Récupérer eventtime de df_properties (et non df_propreties !) ---
eventtime_row = df_properties[df_properties['key'] == "eventtime"]
if eventtime_row.empty:
    print("[SAFE] No 'eventtime' found in df_properties. No aftershocks can be searched.")
    gdf_aftershock = None
else:
    eventtime_str = eventtime_row["value"].iloc[0]
    eventtime = datetime.fromisoformat(eventtime_str.replace("Z", "+00:00"))

    # Fenêtre d'aftershock (corrigée)
    aftershock_start = eventtime + timedelta(seconds=1)
    aftershock_end = eventtime + timedelta(days=days_after)

    # Limiter au "maintenant" (UTC-aware)
    now = datetime.now(timezone.utc)
    if aftershock_end > now:
        print(f"[INFO] Date de fin ajustée à maintenant ({now}) car dans le futur.")
        aftershock_end = now

    print(f"Recherche aftershocks de {aftershock_start} à {aftershock_end}")

    # Recherche des aftershocks
    aftershocks = search(
        minlongitude=minx,
        minlatitude=miny,
        maxlongitude=maxx,
        maxlatitude=maxy,
        starttime=aftershock_start,
        endtime=aftershock_end,
    )

    # DataFrame
    data = [{c: getattr(e, c, None) for c in cols} for e in aftershocks]
    df_aftershock = pd.DataFrame(data)

    if len(df_aftershock) == 0:
        print("Aucun aftershock trouvé dans la période spécifiée.")
        gdf_aftershock = None
    else:
        # Ajouter la colonne geometry avec Point(lon, lat)
        df_aftershock["geometry"] = df_aftershock.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)
        # Convertir en GeoDataFrame
        gdf_aftershock = gpd.GeoDataFrame(df_aftershock, geometry="geometry", crs="EPSG:4326")
        print(gdf_aftershock.head())


Recherche aftershocks de 2025-03-28 00:00:01+00:00 à 2025-04-27 00:00:00+00:00
           id                             time  magnitude  depth  latitude  \
0  us7000pn9s 2025-03-28 06:20:52.715000+00:00        7.7   10.0   22.0110   
1  us7000pn9z 2025-03-28 06:32:04.777000+00:00        6.7   10.0   21.6975   
2  us7000pncy 2025-03-28 06:39:14.645000+00:00        4.8   10.0   19.8979   
3  us7000pncv 2025-03-28 06:42:24.760000+00:00        4.9   10.0   21.8377   
4  us7000pnb6 2025-03-28 06:45:44.906000+00:00        4.9   10.0   19.1284   

   longitude magtype   gap   rms horizontal_error vertical_error  \
0    95.9363    None  None  None             None           None   
1    95.9690    None  None  None             None           None   
2    95.8026    None  None  None             None           None   
3    95.8747    None  None  None             None           None   
4    96.2075    None  None  None             None           None   

                  geometry  
0   POINT (95.